# QAOA mapping from the QUBO (IBM QISKIT)

The most known variational algorithm for combinatorial optimization is the Quantum Approximate Optimization Algorithm (QAOA) [[1]](https://arxiv.org/abs/1411.4028). The is a hybrid algorithm, i.e. it uses both a quantum component (to compute the eigenvalue of the Hamiltonian for a given state) and a classical component to change the state for the next iteration.

In the following we use the [IBM qiskit framework](https://qiskit.org/).

In [1]:
from qiskit import BasicAer
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit.optimization import QuadraticProgram

Here we will tackle the problfrom described in the notebook **Green_optimization_QuantumAnnealing.ipynb**. We seek the minimum function value, and corresponding configuration of variables, for that we can use the quadratic and linear terms obatained from the **qbm** model.

In [1]:
# Assume 4 steps in the schedule
n_schedules = 4

# define binary variables
schedules = [f's_{i}' for i in range(n_schedules)]

# define a syntetic load demand schedule
demand_schedule = [15, 12, 20, 5]  


# Gas 'g', Solar 's', Wind 'w'
sources = ['g', 's', 'w']

# define syntetic cost of usage (total operating costs)
cost_usage = [30, 5, 10]

# define the cost to switch off/on one of the energy sources
cost_switch = [5, 1, 1]

# define the carbon emission cost per kWh (proxied by cost/kWh of the Solar Renewable Energy Certificates (RECs) in US)
cost_emission = [5, 1, 2]

# define the max power generation per kWh per source and plant
capacity = [15, 7, 10]

We first define the QUBO problem for our case:

In [3]:
qubo = QuadraticProgram()
for s in schedules:
    for alpha in sources:
        qubo.binary_var(s+alpha)

and use the previous information from the notebook **Green_optimization_QuantumAnnealing.ipynb**

In [4]:
#dictinary with the variables and respective coefficients for the linear terms:
d_linear = {'s_0g': -33225.0, 's_0s': -24060.0, 's_0w': -29820.0, 's_1g': -19830.0, 
            's_1s': -17778.0, 's_1w': -20856.0, 's_2g': -55550.0, 's_2s': -34530.0, 
            's_2w': -44760.0, 's_3g': 11425.0, 's_3s': -3120.0, 's_3w': 60.0}

In [5]:
#dictinary with the variables and respective coefficients for the quadritic terms:
d_quadratic = {('s_0s', 's_0g'): 31500.0, ('s_0w', 's_0g'): 45000.0, ('s_0w', 's_0s'): 21000.0, ('s_1g', 's_0s'): 6.0, 
               ('s_1g', 's_0w'): 6.0, ('s_1s', 's_0g'): 6.0, ('s_1s', 's_0w'): 2.0, ('s_1s', 's_1g'): 31500.0, 
               ('s_1w', 's_0g'): 6.0, ('s_1w', 's_0s'): 2.0, ('s_1w', 's_1g'): 45000.0, ('s_1w', 's_1s'): 21000.0, 
               ('s_2g', 's_1s'): 6.0, ('s_2g', 's_1w'): 6.0, ('s_2s', 's_1g'): 6.0, ('s_2s', 's_1w'): 2.0, 
               ('s_2s', 's_2g'): 31500.0, ('s_2w', 's_1g'): 6.0, ('s_2w', 's_1s'): 2.0, ('s_2w', 's_2g'): 45000.0, 
               ('s_2w', 's_2s'): 21000.0, ('s_3g', 's_2s'): 6.0, ('s_3g', 's_2w'): 6.0, ('s_3s', 's_2g'): 6.0, 
               ('s_3s', 's_2w'): 2.0, ('s_3s', 's_3g'): 31500.0, ('s_3w', 's_2g'): 6.0, ('s_3w', 's_2s'): 2.0, 
               ('s_3w', 's_3g'): 45000.0, ('s_3w', 's_3s'): 21000.0}

In [9]:
qubo.minimize(linear=[v for v in d_linear.values()], 
              quadratic=d_quadratic)

In [10]:
print(qubo.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 33225 s_0g - 24060 s_0s - 29820 s_0w - 19830 s_1g - 17778 s_1s
      - 20856 s_1w - 55550 s_2g - 34530 s_2s - 44760 s_2w + 11425 s_3g
      - 3120 s_3s + 60 s_3w + [ 63000 s_0g*s_0s + 90000 s_0g*s_0w + 12 s_0g*s_1s
      + 12 s_0g*s_1w + 42000 s_0s*s_0w + 12 s_0s*s_1g + 4 s_0s*s_1w
      + 12 s_0w*s_1g + 4 s_0w*s_1s + 63000 s_1g*s_1s + 90000 s_1g*s_1w
      + 12 s_1g*s_2s + 12 s_1g*s_2w + 42000 s_1s*s_1w + 12 s_1s*s_2g
      + 4 s_1s*s_2w + 12 s_1w*s_2g + 4 s_1w*s_2s + 63000 s_2g*s_2s
      + 90000 s_2g*s_2w + 12 s_2g*s_3s + 12 s_2g*s_3w + 42000 s_2s*s_2w
      + 12 s_2s*s_3g + 4 s_2s*s_3w + 12 s_2w*s_3g + 4 s_2w*s_3s
      + 63000 s_3g*s_3s + 90000 s_3g*s_3w + 42000 s_3s*s_3w ]/2
Subject To

Bounds
 0 <= s_0g <= 1
 0 <= s_0s <= 1
 0 <= s_0w <= 1
 0 <= s_1g <= 1
 0 <= s_1s <= 1
 0 <= s_1w <= 1
 0 <= s_2g <= 1
 0 <= s_2s <= 1
 0 <= s_2w <= 1
 0 <= s_3g <= 1
 0 <= s_3s <= 1
 0 <= s_3w <

Sometimes an QuadraticProgram might also directly be given in the form of an Operator. For such cases, Qiskit also provides a converter from an Operator back to a QuadraticProgram, which we illustrate in the following.

In [11]:
op, offset = qubo.to_ising()
print('offset: {}'.format(offset))
print('operator:')
print(op)

offset: -38501.0
operator:
SummedOp([
  -2515.5 * IIIIIIIIIIIZ,
  -1097.0 * IIIIIIIIIIZI,
  -1592.0 * IIIIIIIIIZII,
  -9216.0 * IIIIIIIIZIII,
  -4240.0 * IIIIIIIZIIII,
  -6076.0 * IIIIIIZIIIII,
  8644.0 * IIIIIZIIIIII,
  4136.0 * IIIIZIIIIIII,
  5876.0 * IIIZIIIIIIII,
  -24840.5 * IIZIIIIIIIII,
  -11567.0 * IZIIIIIIIIII,
  -16532.0 * ZIIIIIIIIIII,
  7875.0 * IIIIIIIIIIZZ,
  11250.0 * IIIIIIIIIZIZ,
  5250.0 * IIIIIIIIIZZI,
  1.5 * IIIIIIIIZIZI,
  1.5 * IIIIIIIIZZII,
  1.5 * IIIIIIIZIIIZ,
  0.5 * IIIIIIIZIZII,
  7875.0 * IIIIIIIZZIII,
  1.5 * IIIIIIZIIIIZ,
  0.5 * IIIIIIZIIIZI,
  11250.0 * IIIIIIZIZIII,
  5250.0 * IIIIIIZZIIII,
  1.5 * IIIIIZIZIIII,
  1.5 * IIIIIZZIIIII,
  1.5 * IIIIZIIIZIII,
  0.5 * IIIIZIZIIIII,
  7875.0 * IIIIZZIIIIII,
  1.5 * IIIZIIIIZIII,
  0.5 * IIIZIIIZIIII,
  11250.0 * IIIZIZIIIIII,
  5250.0 * IIIZZIIIIIII,
  1.5 * IIZIZIIIIIII,
  1.5 * IIZZIIIIIIII,
  1.5 * IZIIIZIIIIII,
  0.5 * IZIZIIIIIIII,
  7875.0 * IZZIIIIIIIII,
  1.5 * ZIIIIZIIIIII,
  0.5 * ZIIIZIIIIIII,
 

In [12]:
qp=QuadraticProgram()
qp.from_ising(op, offset, linear=True)
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 33225 x_0 - 24060 x_1 - 29820 x_2 - 19830 x_3 - 17778 x_4 - 20856 x_5
      - 55550 x_6 - 34530 x_7 - 44760 x_8 + 11425 x_9 - 3120 x_10 + 60 x_11 + [
      63000 x_0*x_1 + 90000 x_0*x_2 + 12 x_0*x_4 + 12 x_0*x_5 + 42000 x_1*x_2
      + 12 x_1*x_3 + 4 x_1*x_5 + 12 x_2*x_3 + 4 x_2*x_4 + 63000 x_3*x_4
      + 90000 x_3*x_5 + 12 x_3*x_7 + 12 x_3*x_8 + 42000 x_4*x_5 + 12 x_4*x_6
      + 4 x_4*x_8 + 12 x_5*x_6 + 4 x_5*x_7 + 63000 x_6*x_7 + 90000 x_6*x_8
      + 12 x_6*x_10 + 12 x_6*x_11 + 42000 x_7*x_8 + 12 x_7*x_9 + 4 x_7*x_11
      + 12 x_8*x_9 + 4 x_8*x_10 + 63000 x_9*x_10 + 90000 x_9*x_11
      + 42000 x_10*x_11 ]/2
Subject To

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1
 0 <= x_5 <= 1
 0 <= x_6 <= 1
 0 <= x_7 <= 1
 0 <= x_8 <= 1
 0 <= x_9 <= 1
 0 <= x_10 <= 1
 0 <= x_11 <= 1

Binaries
 x_0 x_1 x_2 x_3 x_4 x_5 x_6 x_7 x_8 x_9 x_10 x_11
End



This converter allows, for instance, to translate an Operator to a QuadraticProgram and then solve the problem with other algorithms that are not based on the Ising Hamiltonian representation, such as the GroverOptimizer.

## Solving a QUBO with the MinimumEigenOptimizer and QAOA

In [13]:
aqua_globals.random_seed = 42
quantum_instance = QuantumInstance(BasicAer.get_backend('statevector_simulator'),
                                   seed_simulator=aqua_globals.random_seed,
                                   seed_transpiler=aqua_globals.random_seed)
qaoa_mes = QAOA(quantum_instance=quantum_instance, initial_point=[0., 0.])
exact_mes = NumPyMinimumEigensolver()

In [14]:
qaoa = MinimumEigenOptimizer(qaoa_mes)   # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

We first use the MinimumEigenOptimizer based on the classical exact NumPyMinimumEigensolver to get the optimal benchmark

In [15]:
exact_result = exact.solve(qubo)
print(exact_result)

optimal function value: -115761.0
optimal value: [1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0.]
status: SUCCESS


Next we apply the MinimumEigenOptimizer based on QAOA to the same problem:

In [16]:
qaoa_result = qaoa.solve(qubo)
print(qaoa_result)

optimal function value: -115761.0
optimal value: [1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0.]
status: SUCCESS


which can be translated to the schedule:

's_0g': 1, 's_0s': 0, 's_0w': 0, 's_1g': 0, 's_1s': 0, 's_1w': 1, 's_2g': 1, 's_2s': 1, 's_2w': 0, 's_3g': 0, 's_3s': 1, 's_3w': 0

we can compare with the D-wave's suggested one:

's_0g': 1, 's_0s': 0, 's_0w': 0, 's_1g': 0, 's_1s': 0, 's_1w': 1, 's_2g': 1, 's_2s': 1, 's_2w': 0, 's_3g': 0, 's_3s': 1, 's_3w': 0 

and it ields **the same value as found on the D-wave Exact Solver**.